<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_Python/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Xavier initialization
        self.Wxh = np.random.randn(hidden_size, input_size) / np.sqrt(input_size)
        self.Whh = np.random.randn(hidden_size, hidden_size) / np.sqrt(hidden_size)
        self.Why = np.random.randn(output_size, hidden_size) / np.sqrt(hidden_size)

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def softmax(self, x):
        e = np.exp(x - np.max(x))
        return e / np.sum(e)

    def forward(self, inputs):
        """
        inputs: list of one-hot vectors (input_size x 1)
        returns: (outputs, hidden_states)
        """
        h = np.zeros((self.hidden_size, 1))
        hs = {-1: h}
        ys = {}
        ps = {}

        for t in range(len(inputs)):
            x = inputs[t]
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            y = self.Why @ h + self.by
            p = self.softmax(y)

            hs[t] = h
            ys[t] = y
            ps[t] = p

        return ys, ps, hs

    def backward(self, inputs, targets, ps, hs, learning_rate=1e-2):
        """
        Cross-entropy loss + BPTT
        targets: list of integer class indices
        """
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)

        dh_next = np.zeros_like(hs[0])
        loss = 0

        for t in reversed(range(len(inputs))):
            p = ps[t]
            target_idx = targets[t]

            # Cross-entropy loss
            loss += -np.log(p[target_idx, 0])

            # Gradient of softmax + CE
            dy = p.copy()
            dy[target_idx] -= 1  # dL/dy

            dWhy += dy @ hs[t].T
            dby += dy

            dh = self.Why.T @ dy + dh_next
            dh_raw = (1 - hs[t] ** 2) * dh

            dbh += dh_raw
            dWxh += dh_raw @ inputs[t].T
            dWhh += dh_raw @ hs[t-1].T

            dh_next = self.Whh.T @ dh_raw

        # Clip exploding gradients
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)

        # Update parameters
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh  -= learning_rate * dbh
        self.by  -= learning_rate * dby

        return loss


# Example: vocabulary of 5 symbols
vocab_size = 5
hidden_size = 16
rnn = RNN(vocab_size, hidden_size, vocab_size)

def one_hot(idx, size):
    v = np.zeros((size, 1))
    v[idx] = 1
    return v

for epoch in range(2000):
    # Random sequence
    seq_len = 4
    inputs = [one_hot(np.random.randint(vocab_size), vocab_size) for _ in range(seq_len)]
    targets = [np.random.randint(vocab_size) for _ in range(seq_len)]

    ys, ps, hs = rnn.forward(inputs)
    loss = rnn.backward(inputs, targets, ps, hs, learning_rate=1e-2)

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 5.7268
Epoch 200, Loss: 5.9601
Epoch 400, Loss: 6.5577
Epoch 600, Loss: 6.2616
Epoch 800, Loss: 6.1602
Epoch 1000, Loss: 6.1239
Epoch 1200, Loss: 6.4440
Epoch 1400, Loss: 6.4738
Epoch 1600, Loss: 6.5945
Epoch 1800, Loss: 6.1128
